In [1]:
import numpy as np

In [103]:
matrix = np.array([[33, 10, 20, 26.5],
                   [50, 67, 11.5, 25], 
                   [23.5, 35, 40, 58.5]])



In [111]:
eps = 0.1

In [106]:
np.random.seed = 42
matrix = np.random.randint(-10, 10, size=(100, 100))

# Браун-Робинсон

In [112]:
%%time 
prev_P = np.zeros((1, matrix.shape[0]))
prev_Q = np.zeros((matrix.shape[1],1))
i = np.argmax(np.mean(matrix, axis=1))
j = np.argmin(np.mean(matrix, axis=0))

prev_P[0,i] = 1
prev_Q[j] = 1

best_P = prev_P
best_Q = prev_Q

alpha_k = np.min(matrix[i])
beta_k = np.max(matrix[:,j])

k = 1


while True:        
    i_new = np.argmax(np.dot(matrix, prev_Q))
    j_new = np.argmin(np.dot(prev_P, matrix))

    Q_k = k * prev_Q / (k + 1)
    Q_k[j_new] += 1 / (k + 1)

    P_k = k * prev_P / (k + 1)

    P_k[0,i_new] += 1 / (k + 1)

    alpha_P = np.min(np.dot(P_k, matrix))
    beta_Q = np.max(np.dot(matrix, Q_k))

    if alpha_k < alpha_P:
        alpha_k = alpha_P
        best_P = P_k
    if beta_k > beta_Q:
        beta_k = beta_Q
        best_Q = Q_k

    prev_P = P_k
    prev_Q = Q_k

    delta = beta_k - alpha_k
    if delta <= 2*eps:
        break
    else:
        k+=1
print(k)

31514
CPU times: user 5.56 s, sys: 4.57 s, total: 10.1 s
Wall time: 5.23 s


In [95]:
I = (alpha_k + beta_k)/2

In [79]:
  print('Игрок 1:')
  print(pd.DataFrame(best_P, columns=[f'Сратегия {i+1}' for i in range(len(best_P[0]))]))
  print(f'Игрок 2:')
  print(pd.DataFrame(best_Q.reshape(1,-1), columns=[f'Сратегия {i+1}' for i in range(len((best_Q.reshape(1,-1)[0])))]))
  print(f'Цена игры: {round(I, 4)}')

Игрок 1:
   Сратегия 1  Сратегия 2  Сратегия 3  ...  Сратегия 98  Сратегия 99  Сратегия 100
0    0.005205     0.00067         0.0  ...          0.0     0.038169      0.003391

[1 rows x 100 columns]
Игрок 2:
   Сратегия 1  Сратегия 2  Сратегия 3  ...  Сратегия 98  Сратегия 99  Сратегия 100
0    0.015679    0.009559    0.001608  ...     0.000536     0.001876           0.0

[1 rows x 100 columns]
Цена игры: -0.3746


# Сведение к ЗЛП

In [80]:
!pip install pulp

In [81]:
import pandas as pd
from pulp import *

In [82]:

def clean_strg(A):
  low_value = np.max(np.min(A, axis=1))
  high_value = np.min(np.max(A, axis=0))
  print(low_value, ' ', high_value)
  if np.isclose(low_value, high_value):
    game_price = low_value
  else:
    game_price = None
  return game_price

In [96]:
%%time
if clean_strg(matrix):
  print('Игра решается в чистых стратегиях. Цена игры:', clean_strg(matrix))
  strategy_first = np.argmax(np.min(matrix, axis=1))
  strategy_second = np.argmin(np.max(matrix, axis=0))
  print('Стратегия для первого игрока: ', strategy_first+1, '. Для второго: ', strategy_second+1)
else: 
  print('Игра не решается в чистых стратегиях.')
  if (matrix <= 0).any():
    a = np.abs(matrix.min()) + 1
    matrix += a
  else:
    a = 0

  LP1 = pulp.LpProblem('1player', 1)
  vars1 = LpVariable.dicts('1p', range(0, matrix.T.shape[1]), cat='Continuous', lowBound=0)
  LP1 += lpSum([vars1[var] for var in vars1]), "P1"
  for i in range(0, matrix.T.shape[0]):
    LP1+= lpSum([matrix.T[i][j]*vars1[j] for j in range(0, matrix.T.shape[1])]) >= 1
 # print(LP1)
  slvr = PULP_CBC_CMD()
  s1 = LP1.solve(slvr)
  #print(LpStatus[s1])
  U = np.array([])
  for u in LP1.variables():
    if u.varValue != None:
      U = np.append(U, u.varValue)
    #print(u.name, '=', u.varValue)

  LP2 = pulp.LpProblem('2player', -1)
  vars2 = LpVariable.dicts('2p', range(0, matrix.shape[1]), cat='Continuous', lowBound=0)
  LP2 += lpSum([vars2[var] for var in vars2]), "P2"
  for i in range(0, matrix.shape[0]):
    LP2+= lpSum([matrix[i][j]*vars2[j] for j in range(0, matrix.shape[1])]) <= 1
  #print(LP2)
  s2 = LP2.solve()
  V = np.array([])
  for v in LP2.variables():
    if v.varValue != None:
      V = np.append(V, v.varValue)
    #print(v.name, '=', v.varValue)

  I1 = 1 / sum(U)
  I2 = 1 / sum(V)
  p = np.round(U * I1, 2)
  q = np.round(V * I2, 2)
  I = np.round(np.mean([I1, I2]), 4) - a
  # print('Игрок 1:')
  # print(pd.DataFrame([p], columns=[f'Сратегия {i+1}' for i in range(len(p))],
  #                     index=['']).T, '\n')
  # print(f'Игрок 2:')
  # print(pd.DataFrame([q], columns=[f'Сратегия {i+1}' for i in range(len(q))],
  #                     index=['']).T, '\n')
  # print(f'Цена игры: {I}')


-28   27
Игра не решается в чистых стратегиях.
CPU times: user 582 ms, sys: 13.6 ms, total: 595 ms
Wall time: 642 ms
